# How to use the backend calibrations maintained by IBM to calculate the Cross Cross Resonance pulse parameters

## 1.  Collection of Provided Values

In [58]:
import pandas as pd
import os
csv_path = os.getcwd() + "\\ibmq_casablanca_calibrations_2021_04_06.csv"
df = pd.read_csv(csv_path)
print(df.columns)
df

Index(['Qubit', 'T1 (us)', 'T2 (us)', 'Frequency (GHz)', 'Anharmonicity (GHz)',
       'Readout assignment error ', 'Prob meas0 prep1 ', 'Prob meas1 prep0 ',
       'Readout length (ns)', 'ID error ', '√x (sx) error ',
       'Single-qubit Pauli-X error ', 'CNOT error ', 'Gate time (ns)'],
      dtype='object')


,Qubit,T1 (us),T2 (us),Frequency (GHz),Anharmonicity (GHz),Readout assignment error,Prob meas0 prep1,Prob meas1 prep0,Readout length (ns),ID error,√x (sx) error,Single-qubit Pauli-X error,CNOT error,Gate time (ns)
0,Q0,98.23,57.33,4.822,-0.33990,0.0398,0.0560,0.0236,5351.111,0.000272,0.000272,0.000272,0_1:7.774e-3,0_1:497.778
1,Q1,125.07,125.85,4.760,-0.32644,0.0143,0.0162,0.0124,5351.111,0.000182,0.000182,0.000182,1_3:5.189e-3; 1_0:7.774e-3; 1_2:1.296e-2,1_3:355.556; 1_0:462.222; 1_2:398.222
2,Q2,107.36,131.63,4.907,-0.33862,0.0167,0.0186,0.0148,5351.111,0.002543,0.002543,0.002543,2_1:1.296e-2,2_1:433.778
3,Q3,93.69,156.47,4.879,-0.33864,0.0179,0.0282,0.0076,5351.111,0.000195,0.000195,0.000195,3_5:7.657e-3; 3_1:5.189e-3,3_5:433.778; 3_1:320
4,Q4,123.99,52.08,4.871,-0.33889,0.0242,0.0342,0.0142,5351.111,0.000316,0.000316,0.000316,4_5:7.640e-3,4_5:398.222
5,Q5,59.04,10.46,4.964,-0.32184,0.0189,0.0298,0.0080,5351.111,0.000341,0.000341,0.000341,5_6:1.522e-2; 5_4:7.640e-3; 5_3:7.657e-3,5_6:341.333; 5_4:362.667; 5_3:398.222
6,Q6,102.03,142.38,5.177,-0.33499,0.0184,0.0276,0.0092,5351.111,0.000222,0.000222,0.000222,6_5:1.522e-2,6_5:305.778


In [59]:
from qiskit import pulse
import numpy as np
import matplotlib.pyplot as plt
from qiskit.pulse import Play, Schedule, DriveChannel, ControlChannel, Waveform
from qiskit.pulse.library import drag, GaussianSquare
from qiskit import QuantumCircuit, Aer, IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')

backend_config = backend.configuration()
backend_defaults = backend.defaults()

print(backend_defaults)

C:\ProgramData\Anaconda3\envs\walter\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2021-04-06 18:03:26,006: Credentials are already in use. The existing account in the session will be replaced.


<PulseDefaults(<InstructionScheduleMap(1Q instructions:
  q0: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
  q1: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
  q2: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
  q3: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
  q4: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
  q5: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
  q6: {'rz', 'u1', 'sx', 'x', 'id', 'u3', 'measure', 'u2'}
Multi qubit instructions:
  (0, 1): {'cx'}
  (1, 0): {'cx'}
  (1, 2): {'cx'}
  (1, 3): {'cx'}
  (2, 1): {'cx'}
  (3, 1): {'cx'}
  (3, 5): {'cx'}
  (4, 5): {'cx'}
  (5, 3): {'cx'}
  (5, 4): {'cx'}
  (5, 6): {'cx'}
  (6, 5): {'cx'}
  (0, 1, 2, 3, 4, 5, 6): {'measure'}
)>Qubit Frequencies [GHz]
[4.8220678240062105, 4.759858136331707, 4.907299973647209, 4.878972802917337, 4.870905406405776, 4.963897856016808, 5.177097674634709]
Measurement Frequencies [GHz]
[7.284054663, 7.385438514, 7.32318003, 7.230756417, 7.280310863, 7.146993

In [60]:
dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns") 

Sampling time: 0.2222222222222222 ns


In [61]:
gates_config = backend_config.gates 
print(gates_config )

[GateConfig(id, [], gate id q { U(0, 0, 0) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(rz, ['theta'], gate rz(theta) q { U(0, 0, theta) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(sx, [], gate sx q { U(pi/2, 3*pi/2, pi/2) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(x, [], gate x q { U(pi, 0, pi) q; }, [[0], [1], [2], [3], [4], [5], [6]]), GateConfig(cx, [], gate cx q0, q1 { CX q0, q1; }, [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]), GateConfig(reset, None, None)]


In [62]:
channels  = backend_config.channels  
print(channels)

{'acquire0': {'operates': {'qubits': [0]}, 'purpose': 'acquire', 'type': 'acquire'}, 'acquire1': {'operates': {'qubits': [1]}, 'purpose': 'acquire', 'type': 'acquire'}, 'acquire2': {'operates': {'qubits': [2]}, 'purpose': 'acquire', 'type': 'acquire'}, 'acquire3': {'operates': {'qubits': [3]}, 'purpose': 'acquire', 'type': 'acquire'}, 'acquire4': {'operates': {'qubits': [4]}, 'purpose': 'acquire', 'type': 'acquire'}, 'acquire5': {'operates': {'qubits': [5]}, 'purpose': 'acquire', 'type': 'acquire'}, 'acquire6': {'operates': {'qubits': [6]}, 'purpose': 'acquire', 'type': 'acquire'}, 'd0': {'operates': {'qubits': [0]}, 'purpose': 'drive', 'type': 'drive'}, 'd1': {'operates': {'qubits': [1]}, 'purpose': 'drive', 'type': 'drive'}, 'd2': {'operates': {'qubits': [2]}, 'purpose': 'drive', 'type': 'drive'}, 'd3': {'operates': {'qubits': [3]}, 'purpose': 'drive', 'type': 'drive'}, 'd4': {'operates': {'qubits': [4]}, 'purpose': 'drive', 'type': 'drive'}, 'd5': {'operates': {'qubits': [5]}, 'purp

In [63]:
system_hamiltonian = backend_config.hamiltonian 
print(system_hamiltonian)

{'description': 'Qubits are modeled as Duffing oscillators. In this case, the system includes higher energy states, i.e. not just |0> and |1>. The Pauli operators are generalized via the following set of transformations:\n\n$(\\mathbb{I}-\\sigma_{i}^z)/2 \\rightarrow O_i \\equiv b^\\dagger_{i} b_{i}$,\n\n$\\sigma_{+} \\rightarrow b^\\dagger$,\n\n$\\sigma_{-} \\rightarrow b$,\n\n$\\sigma_{i}^X \\rightarrow b^\\dagger_{i} + b_{i}$.\n\nQubits are coupled through resonator buses. The provided Hamiltonian has been projected into the zero excitation subspace of the resonator buses leading to an effective qubit-qubit flip-flop interaction. The qubit resonance frequencies in the Hamiltonian are the cavity dressed frequencies and not exactly what is returned by the backend defaults, which also includes the dressing due to the qubit-qubit interactions.\n\nQuantities are returned in angular frequencies, with units 2*pi*GHz.\n\nWARNING: Currently not all system Hamiltonian information is available

In [64]:
system_hamiltonian_params = backend_config.hamiltonian['vars']
print(system_hamiltonian_params)

{'delta0': -2135684895.7375534, 'delta1': -2051069517.9330974, 'delta2': -2127640392.4573247, 'delta3': -2127748155.8240955, 'delta4': -2129301739.398853, 'delta5': -2022190613.7524579, 'delta6': -2104827921.7467928, 'jq0q1': 8155201.12104726, 'jq1q2': 8960302.871474529, 'jq1q3': 8704187.306888234, 'jq3q5': 8755759.807187283, 'jq4q5': 9192358.915922279, 'jq5q6': 10792387.809142636, 'omegad0': 1440298403.757042, 'omegad1': 1042913625.0764529, 'omegad2': 1411010065.6535163, 'omegad3': 1547366729.424079, 'omegad4': 1490460238.439909, 'omegad5': 1082289090.1053412, 'omegad6': 916313586.8622271, 'wq0': 30297945702.019264, 'wq1': 29907070706.45859, 'wq2': 30833475092.342915, 'wq3': 30655490229.419018, 'wq4': 30604801282.190388, 'wq5': 31189090075.265053, 'wq6': 32528664043.098404}


In [65]:
inst_sched_map_instructions = inst_sched_map.instructions
print(inst_sched_map_instructions)

['cx', 'id', 'measure', 'rz', 'sx', 'u1', 'u2', 'u3', 'x']


In [68]:
n_uchannels = backend_config.n_uchannels
print(n_uchannels)

12


In [66]:
u_channel_lo = backend_config.u_channel_lo
print(u_channel_lo)

[[UchannelLO(1, (1+0j))], [UchannelLO(0, (1+0j))], [UchannelLO(2, (1+0j))], [UchannelLO(3, (1+0j))], [UchannelLO(1, (1+0j))], [UchannelLO(1, (1+0j))], [UchannelLO(5, (1+0j))], [UchannelLO(5, (1+0j))], [UchannelLO(3, (1+0j))], [UchannelLO(4, (1+0j))], [UchannelLO(6, (1+0j))], [UchannelLO(5, (1+0j))]]


In [67]:
qubit_lo_range = backend_config.qubit_lo_range 
print(qubit_lo_range )

[[4322067824.00621, 5322067824.00621], [4259858136.331707, 5259858136.331707], [4407299973.647209, 5407299973.647209], [4378972802.917337, 5378972802.917337], [4370905406.405776, 5370905406.405776], [4463897856.016808, 5463897856.016808], [4677097674.634708, 5677097674.634708]]


In [70]:
meas_lo_range = backend_config.meas_lo_range 
print(meas_lo_range )

[[6784054663.0, 7784054663.0], [6885438514.0, 7885438514.0], [6823180030.0, 7823180030.0], [6730756417.0, 7730756417.0], [6780310863.0, 7780310863.0], [6646993373.0, 7646993373.0], [6900652534.0, 7900652534.0]]


In [69]:
meas_levels = backend_config.meas_levels
print(meas_levels)

[1, 2]
